---
title: "Making Large Language Models produce structured output using Guardrails AI"
jupyter: python3
format:
  html:
    code-overflow: wrap
filters:
  - line-highlight
---

In [66]:
from textwrap import TextWrapper

wrapper = TextWrapper(width=80)

In [51]:
import openai

## Language Models are hard to configure and control

Let's start with a very simple prompt asking OpenAI's GPT-3 to add 1+1 and return the answer. In this case, we would like GPT-3 to act as a calculator and return back to us the result. 

Note for those not familiar with how to call OpenAI chat completion: the TLDR is you choose a model, send the prompts as history of system, user and assistant messages, along with somple sampling parameters like (`top_p` or `temperature`) and then you get back a response which you can parse the assistant message from.

In [52]:
#| echo: TRUE
#| eval: FALSE
#| source-line-numbers: 6
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {
            "role": "system", "content": "You are a helpful assistant.",
            "role": "user", "content": "Return only the integer answer, 1+1=",
        }
    ],
    temperature=0,
)

response['choices'][0]["message"]['content']

'The integer answer to 1+1 is 2.'

You can see that even though we explicitly stated "Return only the integer answer" expecting only `2` to be returned, the model chose to return a full sentence string instead, causing all sorts of frustration. 

To resolve this naively, we would then go about a "prompt engineering" journey where we try to find the "right prompt template" to get the model to do what we want. This is a very time consuming process and is not scalable.

So the question becomes, when engineering a system that makes use of a language model as one component, how can we enforce control over the output of the model without spending endless cycles tuning prompts ?

There are several approaches to this probelm. 

In this article, we will do a deep dive into the [guardrails AI](https://guardrailsai.com) approach. The TLDR is the following: guardrails AI
- allows us to declare our output schema using familiar tooling like [pydantic](https://docs.pydantic.dev/latest/)
- provides us with out of the box prompt templates that we can use to get the model to produce the output we want
- if the language model fails to produce the output we want, guardrails has out of the box prompt templates to "re-ask the model" to correct its output


## Guardrails AI deep dive

### De-mystifying the guardrails AI approach with a simple example
We start with the same simple query 

In [53]:
query = """1+1=?"""
print(query)

1+1=?


We now define the desired answer schema using [pydantic]((https://docs.pydantic.dev/latest/). Pydantic relies on python type annotations to define the attributes of a class.

In [54]:
from pydantic import BaseModel, Field


class IntegerAnswer(BaseModel):
    """The answer to a question."""

    value: int = Field(description="The answer to the question.")

We then have to define quite a few things to get guardrails AI to work out of the box:
- the system (instruction) prompt template
- the user prompt template

Call guardrails `Guard.from_pydantic` to 
-  produce a spec of our pydantic model that can be inserted into the prompt templates

In [55]:
import guardrails as gd

system_instructions = """
You are a helpful assistant only capable of communicating with valid JSON, and no other text.
"""

user_prompt = """
${query}

${gr.complete_json_suffix_v2}
"""

guard = gd.Guard.from_pydantic(
    instructions=system_instructions,
    prompt=user_prompt,
    output_class=IntegerAnswer,
)

Let's print the produced system prompt and user prompt to see how the spec is inserted into the templates.

In [56]:
print(f"system prompt", guard.instructions)
print(f"user prompt", guard.prompt)

system prompt 
You are a helpful assistant only capable of communicating with valid JSON, and no other text.

user prompt 
${query}


Given below is XML that describes the information to extract from this document and the tags to extract it into.

<output>
    <integer name="value" description="The answer to the question."/>
</output>


ONLY return a valid JSON object (no other text is necessary), where the key of the field in JSON is the `name` attribute of the corresponding XML, and the value is of the type specified by the corresponding XML's tag. The JSON MUST conform to the XML format, including any types and format requests e.g. requests for lists, objects and specific types. Be correct and concise.

Here are examples of simple (XML, JSON) pairs that show the expected behavior:
- `<string name='foo' format='two-words lower-case' />` => `{'foo': 'example one'}`
- `<list name='bar'><string format='upper-case' /></list>` => `{"bar": ['STRING ONE', 'STRING TWO', etc.]}`
- `<object na

To craft our prompt-templates, we need to get familiarized with Guardrails AI's prompt templating language.

- guardrails relies on a `${var}` syntax
- variables like our `query` can be passed in like so `${query}` 
- constants like `complete_json_suffix_v2`  reference pre-defined prompt templates which we can find in the Guardrails AI [constants.xml](https://github.com/guardrails-ai/guardrails/blob/main/guardrails/constants.xml) file


We can see that Guardrails AI has produced the following xml spec from our pydantic model
```xml
<output>
    <integer name="value" description="The answer to the question."/>
</output>
```


### Inspecting how Guardrail builds the prompt

Let's load the `constants.xml` file and see what the `complete_json_suffix_v2` template looks like:

In [57]:
from pathlib import Path
import xml.etree.ElementTree as ET

tree = ET.parse((Path(gd.__file__).parent / "constants.xml"))
root = tree.getroot()

# Now you can access the elements in the XML file
for child in root:
    if child.tag == "complete_json_suffix_v2":
        print(child.text)


Given below is XML that describes the information to extract from this document and the tags to extract it into.

${output_schema}

ONLY return a valid JSON object (no other text is necessary), where the key of the field in JSON is the `name` attribute of the corresponding XML, and the value is of the type specified by the corresponding XML's tag. The JSON MUST conform to the XML format, including any types and format requests e.g. requests for lists, objects and specific types. Be correct and concise.

Here are examples of simple (XML, JSON) pairs that show the expected behavior:
- `<string name='foo' format='two-words lower-case' />` => `{'foo': 'example one'}`
- `<list name='bar'><string format='upper-case' /></list>` => `{"bar": ['STRING ONE', 'STRING TWO', etc.]}`
- `<object name='baz'><string name="foo" format="capitalize two-words" /><integer name="index" format="1-indexed" /></object>` => `{'baz': {'foo': 'Some String', 'index': 1}}`



This gives us the same prompt as before but we can see `${output_schema}` is not populated with a schema yet.

so we can deduce that the Guardrails's `Guard` is responsible for:
- producing the `output_schema` from the pydantic model
- replacing `${output_schema}` with the produced schema

Let's inspect the `Guard` class to see what it is composed from:

In [58]:
gd.Guard?

Init signature:
gd.Guard(
    rail: guardrails.rail.Rail,
    num_reasks: int = None,
    base_model: Optional[pydantic.main.BaseModel] = None,
)
Docstring:     
The Guard class.

This class is the main entry point for using Guardrails. It is
initialized from one of the following class methods:

- `from_rail`
- `from_rail_string`
- `from_pydantic`
- `from_string`

The `__call__`
method functions as a wrapper around LLM APIs. It takes in an LLM
API, and optional prompt parameters, and returns the raw output from
the LLM and the validated output.
Init docstring: Initialize the Guard.
File:           ~/.pyenv/versions/3.10.8/envs/structured-output-llm-py310/lib/python3.10/site-packages/guardrails/guard.py
Type:           type
Subclasses:     

Every `Guard` object is composed of:

- a `Rail` object
- a setting `num_reasks` for how many attempts to re-ask the model in case of a failure
- the pydantic `base_model`
    

Looking at `Guard.from_pydantic` we can see that it is constructing the `Rail` object from the base_model

In [59]:
%psource gd.Guard.from_pydantic

    @classmethod
    def from_pydantic(
        cls,
        output_class: BaseModel,
        prompt: Optional[str] = None,
        instructions: Optional[str] = None,
        num_reasks: int = None,
    ) -> "Guard":
        """Create a Guard instance from a Pydantic model and prompt."""
        rail = Rail.from_pydantic(
            output_class=output_class, prompt=prompt, instructions=instructions
        )
        return cls(rail, num_reasks=num_reasks, base_model=output_class)


If we construct the `Rail` object directly, we can see that it builds out the `output_schema` and updates the prompts for us

In [60]:
rail = gd.Rail.from_pydantic(
    instructions=system_instructions,
    prompt=user_prompt,
    output_class=IntegerAnswer,
)

print("system prompt", rail.instructions)
print("user prompt", rail.prompt)

system prompt 
You are a helpful assistant only capable of communicating with valid JSON, and no other text.

user prompt 
${query}


Given below is XML that describes the information to extract from this document and the tags to extract it into.

<output>
    <integer name="value" description="The answer to the question."/>
</output>


ONLY return a valid JSON object (no other text is necessary), where the key of the field in JSON is the `name` attribute of the corresponding XML, and the value is of the type specified by the corresponding XML's tag. The JSON MUST conform to the XML format, including any types and format requests e.g. requests for lists, objects and specific types. Be correct and concise.

Here are examples of simple (XML, JSON) pairs that show the expected behavior:
- `<string name='foo' format='two-words lower-case' />` => `{'foo': 'example one'}`
- `<list name='bar'><string format='upper-case' /></list>` => `{"bar": ['STRING ONE', 'STRING TWO', etc.]}`
- `<object na

Let's inspect `gd.Rail.from_pydantic` more closely

In [61]:
%psource gd.Rail.from_pydantic

    @classmethod
    def from_pydantic(
        cls,
        output_class: BaseModel,
        prompt: Optional[str] = None,
        instructions: Optional[str] = None,
        reask_prompt: Optional[str] = None,
        reask_instructions: Optional[str] = None,
    ):
        xml = generate_xml_code(
            output_class=output_class,
            prompt=prompt,
            instructions=instructions,
            reask_prompt=reask_prompt,
            reask_instructions=reask_instructions,
        )
        return cls.from_xml(xml)


It relies on a core function called `generate_xml_code` that will produce XML code from the pydantic model. Here is a the docstring for `generate_xml_code` - note that it calls the generated XML code - the XML RAIL Spec - RAIL is short for "Reliable AI Language"

In [62]:
from guardrails.rail import generate_xml_code

print(generate_xml_code.__doc__)

Generate XML RAIL Spec from a pydantic model and a prompt.

    Parameters: Arguments:
        prompt (str): The prompt for this RAIL spec.
        output_class (BaseModel, optional): The Pydantic model that represents the desired output schema.  Do not specify if using a string schema. Defaults to None.
        instructions (str, optional): Instructions for chat models. Defaults to None.
        reask_prompt (str, optional): An alternative prompt to use during reasks. Defaults to None.
        reask_instructions (str, optional): Alternative instructions to use during reasks. Defaults to None.
        validators (List[Validator], optional): The list of validators to apply to the string schema. Do not specify if using a Pydantic model. Defaults to None.
        description (str, optional): The description for a string schema. Do not specify if using a Pydantic model. Defaults to None.
    


### Using our built Guard object to prompt a language model

Next we use the guard object to call our language model.

#### Happy Path: Our system is able to parse the JSON and return the answer.

In [63]:
import warnings

with warnings.catch_warnings():
    # ignore the GuardrailsAI UserWarning
    warnings.filterwarnings("ignore", category=UserWarning)
    
    raw_llm_output, validated_output = guard(
        llm_api=openai.Completion.create,
        prompt_params={"query": query},
        num_reasks=0,
        engine="text-davinci-003",
        max_tokens=1024,
        temperature=0,
    )

validated_output

{'value': 2}

The guard object's `__call__` will perform the following steps:
- Update the prompt by replacing any remaining `${var}` with the appropriate values provided in `prompt_params`
- Call the language model API
- Parse the returned output using the schema
- If the output doesn't match the schema
    - it will proceed to perfrom a corrective action
        - By default the corrective action is to re-prompt the model asking it to resolve the issue 
    - it will repeat this process until the output matches the schema or until a maximum number of attempts is reached.
- The result is returned as both a string and a structured object

#### UnHappy Path: Our system is able to parse the JSON and return the answer.

Now let's mock our language model API to force a failure

In [76]:
from unittest.mock import MagicMock, patch

magic_mock = MagicMock()
magic_mock.return_value = {
    "object": "chat.completion",
    "choices": [
        {
            "index": 0,
            "text": '{"value": "the answer is 2"}',
            "finish_reason": "stop",
        }
    ],
    "usage": {"prompt_tokens": 18, "completion_tokens": 12, "total_tokens": 30},
}

with patch("openai.Completion.create", magic_mock):
    warnings.filterwarnings("ignore", category=UserWarning)
    raw_llm_output, validated_output = guard(
        llm_api=openai.Completion.create,
        prompt_params={"query": query},
        num_reasks=1,
        engine="text-davinci-003",
        max_tokens=1024,
        temperature=0,
    )

print("\n     ".join(wrapper.wrap(repr(validated_output))))

SkeletonReAsk(incorrect_value={'value': 'the answer is 2'},
     fail_results=[FailResult(outcome='fail', metadata=None, error_message='JSON does
     not match schema', fix_value=None)])


### Advanced features in guardrails

#### 

#### UnHappy Path: Introducing Guardrails validators and on-fail actions

So what options do we have when the model fails to produce the output we want? Can we customize the corrective action? Can we do this on a per attribute basis?

The answer is yes. We can define a guardrails validator for each attribute in our schema. The validator will be called on the parsed output and will return a boolean indicating whether the output is valid or not. If the output is not valid, we can define an on-fail action to perform. The on-fail action can be one of the [following actions](https://docs.guardrailsai.com/concepts/output/#specifying-corrective-actions):

- `reask`	Reask the LLM to generate an output that meets the quality criteria. The prompt used for reasking contains information about which quality criteria failed, which is auto-generated by the validator.
- `fix`	Programmatically fix the generated output to meet the quality criteria. E.g. for the formatter two-words, the programatic fix simply takes the first 2 words of the generated string.
- `filter`	Filter the incorrect value. This only filters the field that fails, and will return the rest of the generated output.
- `refrain`	Refrain from returning an output. If a formatter has the corrective action refrain, then on failure there will be a None output returned instead of the JSON.
- `noop`	Do nothing. The failure will still be recorded in the logs, but no corrective action will be taken.
- `exception`	Raise an exception when validation fails.
- `fix_reask`	First, fix the generated output deterministically, and then rerun validation with the deterministically fixed output. If validation fails, then perform reasking.

Let's try out the `ValidRange` validator with an `exception` on_fail action

In [77]:
from guardrails.validators import ValidRange

class Answer(BaseModel):
    value: int = Field(
        description="The answer to the question.",
        validators=[ValidRange(min=0, on_fail="exception")],
    )


guard = gd.Guard.from_pydantic(
    output_class=Answer, prompt=user_prompt, instructions=system_instructions
)

If the language model returns a value that is not a positive integer, an exception will be raised. Let's mock the model to return a negative integer.

In [79]:
magic_mock.return_value = {
    "id": "chatcmpl-8CazZKUCp8KbiUt49J5x7eINiMlvl",
    "choices": [
        {
            "index": 0,
            "text": '{"value": "-2"}',
            "finish_reason": "stop",
        }
    ],
    "usage": {"prompt_tokens": 18, "completion_tokens": 12, "total_tokens": 30},
}

with patch("openai.Completion.create", magic_mock):
    try:
        raw_llm_output, validated_output = guard(
            llm_api=openai.Completion.create,
            prompt_params={"query": query},
            num_reasks=1,
            engine="text-davinci-003",
            max_tokens=1024,
            temperature=0,
        )
    except Exception as e:
        print(type(e))

<class 'guardrails.validators.ValidatorError'>


As you can see a ValidationError is now raised.

Next we inspect the `re-ask` on-fail action behavior by mocking our model to first return an invalid answer, and then a valid answer.

In [94]:
class Answer(BaseModel):
    value: int = Field(
        description="The answer to the question.",
        validators=[ValidRange(min=0, max=10, on_fail="reask")],
    )


guard = gd.Guard.from_pydantic(
    output_class=Answer, prompt=user_prompt, instructions=system_instructions
)

magic_mock = MagicMock()
magic_mock.side_effect = [
    {
        "id": "chatcmpl-8CazZKUCp8KbiUt49J5x7eINiMlvl",
        "choices": [
            {
                "index": 0,
                "text": '{"value": "-2"}',
                "finish_reason": "stop",
            }
        ],
        "usage": {"prompt_tokens": 18, "completion_tokens": 12, "total_tokens": 30},
    },
    {
        "id": "chatcmpl-8CazZKUCp8KbiUt49J5x7eINiMlvl",
        "choices": [
            {
                "index": 0,
                "text": '{"value": "2"}',
                "finish_reason": "stop",
            }
        ],
        "usage": {"prompt_tokens": 18, "completion_tokens": 12, "total_tokens": 30},
    },
]


with patch("openai.Completion.create", magic_mock):
    raw_llm_output, validated_output = guard(
        llm_api=openai.Completion.create,
        prompt_params={"query": query},
        num_reasks=1,
        engine="text-davinci-003",
        max_tokens=1024,
        temperature=0,
    )
validated_output

{'value': 2}

Reasking resolved the issue and returned the correct value ! Lets inpsect the reask prompt that was used

In [99]:
print(magic_mock.call_args[1]["prompt"])


You are a helpful assistant only capable of communicating with valid JSON, and no other text.

ONLY return a valid JSON object (no other text is necessary), where the key of the field in JSON is the `name` attribute of the corresponding XML, and the value is of the type specified by the corresponding XML's tag. The JSON MUST conform to the XML format, including any types and format requests e.g. requests for lists, objects and specific types. Be correct and concise. If you are unsure anywhere, enter `null`.

Here are examples of simple (XML, JSON) pairs that show the expected behavior:
- `<string name='foo' format='two-words lower-case' />` => `{'foo': 'example one'}`
- `<list name='bar'><string format='upper-case' /></list>` => `{"bar": ['STRING ONE', 'STRING TWO', etc.]}`
- `<object name='baz'><string name="foo" format="capitalize two-words" /><integer name="index" format="1-indexed" /></object>` => `{'baz': {'foo': 'Some String', 'index': 1}}`



I was given the following JSON resp

Note that for some reason Guardrails AI doesn't chose to add the initial query 1+1=? to the prompt which is strange given it should be important context for the model.

If instead we used `fix` instead of `reask` then validator will try to coerce a value of 0 or 10 depending on which is closer

In [102]:
class Answer(BaseModel):
    value: int = Field(
        description="The answer to the question.",
        validators=[ValidRange(min=0, max=10, on_fail="fix")],
    )


guard = gd.Guard.from_pydantic(
    output_class=Answer, prompt=user_prompt, instructions=system_instructions
)

magic_mock = MagicMock()
magic_mock.return_value = {
    "id": "chatcmpl-8CazZKUCp8KbiUt49J5x7eINiMlvl",
    "choices": [
        {
            "index": 0,
            "text": '{"value": "-2"}',
            "finish_reason": "stop",
        }
    ],
    "usage": {"prompt_tokens": 18, "completion_tokens": 12, "total_tokens": 30},
}


with patch("openai.Completion.create", magic_mock):
    raw_llm_output, validated_output = guard(
        llm_api=openai.Completion.create,
        prompt_params={"query": query},
        num_reasks=1,
        engine="text-davinci-003",
        max_tokens=1024,
        temperature=0,
    )

print(validated_output)



magic_mock.return_value = {
    "id": "chatcmpl-8CazZKUCp8KbiUt49J5x7eINiMlvl",
    "choices": [
        {
            "index": 0,
            "text": '{"value": "14"}',
            "finish_reason": "stop",
        }
    ],
    "usage": {"prompt_tokens": 18, "completion_tokens": 12, "total_tokens": 30},
}


with patch("openai.Completion.create", magic_mock):
    raw_llm_output, validated_output = guard(
        llm_api=openai.Completion.create,
        prompt_params={"query": query},
        num_reasks=1,
        engine="text-davinci-003",
        max_tokens=1024,
        temperature=0,
    )

print(validated_output)

{'value': '0'}
{'value': '10'}


And if we use `filter` as the corrective action will return an empty dictionary given we only inspect one key named "value"

In [103]:
class Answer(BaseModel):
    value: int = Field(
        description="The answer to the question.",
        validators=[ValidRange(min=0, max=10, on_fail="filter")],
    )


guard = gd.Guard.from_pydantic(
    output_class=Answer, prompt=user_prompt, instructions=system_instructions
)

magic_mock = MagicMock()
magic_mock.return_value = {
    "id": "chatcmpl-8CazZKUCp8KbiUt49J5x7eINiMlvl",
    "choices": [
        {
            "index": 0,
            "text": '{"value": "-2"}',
            "finish_reason": "stop",
        }
    ],
    "usage": {"prompt_tokens": 18, "completion_tokens": 12, "total_tokens": 30},
}


with patch("openai.Completion.create", magic_mock):
    raw_llm_output, validated_output = guard(
        llm_api=openai.Completion.create,
        prompt_params={"query": query},
        num_reasks=1,
        engine="text-davinci-003",
        max_tokens=1024,
        temperature=0,
    )

print(validated_output)

{}


## Time for a more complex example

### Routing between two possible response schemas

For instance if you have a language model that can return more than one possible schema, you can use a choice validator to route between the schemas.


### Complex validators out of the box

For certain cases like checking if the returned output is valid SQL or valid Python, you can use the built-in guardrail validators for these cases.

see the guardrails [validators page](https://docs.guardrailsai.com/api_reference/validators/) for more details.

### Custom validators
Earlier this year there was a [popular video of how ChatGPT couldn't stick to performing legal chess moves](https://www.youtube.com/watch?v=iWhlrkfJrCQ&ab_channel=GothamChess). Guardrails AI has an [example in progress](https://docs.guardrailsai.com/examples/valid_chess_moves/) of how to use custom validators to enforce a legal chess game.

### Flexibility of the guardrails approach
The guardrails approach is very flexible and can be used to validate any kind of language model. 

### Using guardrails to validate a gpt2 model loaded locally


### Using guardrails against the anyscale API

### Weaknesses of Guardrails
- Given that guardrails relies on re-prompts to correct the model, it is not suitable for use cases where the model is expensive to call. 
- Default prompts provided by guardrails might not be optimal for your use case.

### Areas of improvement

- Inheriting validators from pydantic models would be nice but support for it is still lacking.
- Using different models to perform correction
